In this project, we simulate a bank's liquidity risk using two key Basel III metrics:
- **Liquidity Coverage Ratio (LCR)** for short-term liquidity
- **Net Stable Funding Ratio (NSFR)** for long-term funding stability

The balance sheet is constructed using realistic financial items and evaluates how various stress scenarios affect the bank’s liquidity position.


In [11]:
# Import libraries
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Load dataset
file_path = "/kaggle/input/bank-balance-sheet/bank_balance_sheet.csv"  
df = pd.read_csv(file_path)

# Data Validation
required_columns = ['Item', 'Amount (in Millions)', 'Basel_Use']
if not all(col in df.columns for col in required_columns):
    raise ValueError("Dataset is missing required columns.")
if (df['Amount (in Millions)'] < 0).any():
    raise ValueError("Dataset contains negative values.")

# Default outflow rates (Basel III approximation)
default_outflow_rates = {
    "Customer Deposits": 0.10,
    "Interbank Borrowing": 1.00,
    "Short-term Debt": 1.00
}

#  LCR Calculation Function
def calculate_lcr(df, outflow_rates=None, adjustments=None):
    df_copy = df.copy()

    if adjustments:
        for item, adj in adjustments.items():
            if item in df_copy['Item'].values:
                df_copy.loc[df_copy['Item'] == item, 'Amount (in Millions)'] *= adj

    if outflow_rates is None:
        outflow_rates = default_outflow_rates

    hqlas_total = df_copy[df_copy['Basel_Use'] == 'HQLA']['Amount (in Millions)'].sum()

    outflows_total = 0
    for item, rate in outflow_rates.items():
        try:
            value = df_copy[df_copy['Item'] == item]['Amount (in Millions)'].values[0]
        except IndexError:
            value = 0
        outflows_total += value * rate

    try:
        inflow_value = df_copy[df_copy['Item'] == 'Interbank Loans']['Amount (in Millions)'].values[0]
    except IndexError:
        inflow_value = 0
    inflows_total = min(inflow_value, outflows_total * 0.75)

    try:
        obs_drawdown = df_copy[df_copy['Item'] == 'Undrawn Credit Lines']['Amount (in Millions)'].values[0] * 0.30
    except IndexError:
        obs_drawdown = 0
    outflows_total += obs_drawdown

    net_outflows = outflows_total - inflows_total
    lcr = (hqlas_total / net_outflows) * 100 if net_outflows > 0 else float('inf')
    return round(lcr, 2)

# Stress Scenarios
scenarios = {
    "Base Case": {},
    "Deposit Run (30%)": {"Customer Deposits": 1.30},
    "Bond Haircut (20%)": {"Government Bonds": 0.80},
    "Credit Line Drawdown (30%)": {}, 
    "Loan Repayment Delays (20%)": {"Interbank Loans": 0.80},
    "Combined Stress": {
        "Customer Deposits": 1.30,
        "Government Bonds": 0.80,
        "Interbank Loans": 0.80
    }
}

# Run all scenarios
lcr_results = {scenario: calculate_lcr(df, adjustments=adj) for scenario, adj in scenarios.items()}
lcr_plot_df = pd.DataFrame.from_dict(lcr_results, orient='index', columns=['LCR (%)']).reset_index()
lcr_plot_df.rename(columns={'index': 'Scenario'}, inplace=True)

# Plotly Visualization
fig = px.bar(
    lcr_plot_df,
    x='Scenario',
    y='LCR (%)',
    title='Liquidity Coverage Ratio (LCR) Under Stress Scenarios',
    labels={'LCR (%)': 'Liquidity Coverage Ratio (%)'},
    text='LCR (%)',
    color_discrete_sequence=['#1f77b4'] * len(lcr_plot_df)
)

fig.add_hline(y=100, line_dash="dash", line_color="red", annotation_text="Basel III Minimum", annotation_position="bottom right")
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

#  NSFR Calculation
def calculate_nsfr(df):
    asf = df[df['Basel_Use'].isin(['ASF', 'Capital'])]['Amount (in Millions)'].sum()
    rsf = df[df['Basel_Use'] == 'RSF']['Amount (in Millions)'].sum()
    nsfr = (asf / rsf) * 100 if rsf > 0 else float('inf')
    return {
        "Available Stable Funding (ASF)": asf,
        "Required Stable Funding (RSF)": rsf,
        "Net Stable Funding Ratio (NSFR %)": round(nsfr, 2)
    }

# NSFR Output
nsfr_summary = calculate_nsfr(df)
print("📊 NSFR Summary:")
for k, v in nsfr_summary.items():
    print(f"{k}: {v}")


📊 NSFR Summary:
Available Stable Funding (ASF): 1500
Required Stable Funding (RSF): 3100
Net Stable Funding Ratio (NSFR %): 48.39
